# Webscraping with Beautiful Soup
*****


## Intro

In this tutorial, we'll be scraping information on the state senators of Illinois, available [here](http://www.ilga.gov/senate), as well as the list of bills each senator has sponsored (e.g., [here](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True).

## The Tools

1. [Requests](http://docs.python-requests.org/en/latest/user/quickstart/)
2. [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
# import required modules
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import sys

# Part 1: Using Beautiful Soup
*****

## 1.1 Make a Get Request and Read in HTML

We use `requests` library to:
1. make a GET request to the page
2. read in the html of the page

In [ ]:
# make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# read the content of the server’s response
src = req.text

## 1.2 Soup it

Now we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

In [ ]:
# parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# take a look
print(soup.prettify()[:1000])

## 1.3 Find Elements

BeautifulSoup has a number of functions to find things on a page. Like other webscraping tools, Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors


Let's search first for **HTML tags**. 

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

In [ ]:
# find all elements in a certain tag
# these two lines of code are equivilant

soup.find_all("a")

**NB**: Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object. 

These two lines of code are equivalent:

In [ ]:
soup.find_all("a")
soup("a")

That's a lot! Many elements on a page will have the same html tag. For instance, if you search for everything with the `a` tag, you're likely to get a lot of stuff, much of which you don't want. What if we wanted to search for HTML tags ONLY with certain attributes, like particular CSS classes? 

We can do this by adding an additional argument to the `find_all`

In the example below, we are finding all the `a` tags, and then filtering those with `class = "sidemenu"`.

In [ ]:
# Get only the 'a' tags in 'sidemenu' class
soup("a", class_="sidemenu")

Oftentimes a more efficient way to search and find things on a website is by **CSS selector.** For this we have to use a different method, `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use "a.sidemenu" as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [ ]:
# get elements with "a.sidemenu" CSS Selector.
soup.select("a.sidemenu")

## Challenge 1

Find all the `<a>` elements in class `mainmenu`

In [ ]:
# SOLUTION
soup.select("a.mainmenu")

## 1.4 Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Oftentimes this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

In [ ]:
# this is a list
soup.select("a.sidemenu")

# we first want to get an individual tag object
first_link = soup.select("a.sidemenu")[0]

# check out its class
type(first_link)

It's a tag! Which means it has a `text` member:

In [ ]:
print(first_link.text)

Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

You can access a tag’s attributes by treating the tag like a dictionary:

In [ ]:
print(first_link['href'])

## Challenge 2

Find all the `href` attributes (url) from the mainmenu.

In [ ]:
# SOLUTION
[link['href'] for link in soup.select("a.mainmenu")]

# Part 2
*****

Believe it or not, that's all you need to scrape a website. Let's apply these skills to scrape http://www.ilga.gov/senate/default.asp?GA=98

**NB: we're just going to scrape the 98th general assembly.**

Our goal is to scrape information on each senator, including their:
    - name
    - district
    - party

## 2.1 First, make the get request and soup it.

In [ ]:
# make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# read the content of the server’s response
src = req.text
# soup it
soup = BeautifulSoup(src, "lxml")

## 2.2 Find the right elements and text.

Now let's try to get a list of rows in that table. Remember that rows are identified by the `tr` tag.

In [ ]:
# get all tr elements
rows = soup.find_all("tr")
len(rows)

But remember, `find_all` gets all the elements with the `tr` tag. We can use smart CSS selectors to get only the rows we want.

In [ ]:
# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
print(rows[2].prettify())

We can use the `select` method on anything. Let's say we want to find everything with the CSS selector `td.detail` in an item of the list we created above.

In [ ]:
# select only those 'td' tags with class 'detail'
row = rows[2]
detailCells = row.select('td.detail')
detailCells

Most of the time, we're interested in the actual **text** of a website, not its tags. Remember, to get the text of an HTML element, use the `text` member.

In [ ]:
# Keep only the text in each of those cells
rowData = [cell.text for cell in detailCells]

Now we can combine the beautifulsoup tools with our basic python skills to scrape an entire web page.

In [ ]:
# check em out
print(rowData[0]) # Name
print(rowData[3]) # district
print(rowData[4]) # party

## 2.3 Loop it all together

Let's use a for loop to get 'em all!

In [ ]:
# make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')

# read the content of the server’s response
src = req.text

# soup it
soup = BeautifulSoup(src, "lxml")

# Create empty list to store our data
members = []

# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# loop through all rows
for row in rows:
    # select only those 'td' tags with class 'detail'
    detailCells = row.select('td.detail')
    
    # get rid of junk rows
    if len(detailCells) is not 5: 
        continue
        
    # Keep only the text in each of those cells
    rowData = [cell.text for cell in detailCells]
    
    # Collect information
    name = rowData[0]
    district = int(rowData[3])
    party = rowData[4]
    
    # Store in a tuple
    tup = (name,district,party)
    
    # Append to list
    members.append(tup)

In [ ]:
len(members)

In [ ]:
members[1]

## Challege 3: Get HREF element pointing to members' bills. 

The code above retrieves information on:  

    - the senator's name
    - their district number
    - and their party

We now want to retrieve the URL for each senator's list of bills. The format for the list of bills for a given senator is:

http://www.ilga.gov/senate/SenatorBills.asp + ? + GA=98 + &MemberID=**_memberID_** + &Primary=True

to get something like:

http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True

You should be able to see that, unfortunately, _memberID_ is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips: 

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. (See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details).
* NOTE: There are a _lot_ of different ways to use BeautifulSoup to get things done; whatever you need to do to pull that HREF out is fine. Posting on the etherpad is recommended for discussing different strategies.

In [ ]:
# SOLUTION

# make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')

# read the content of the server’s response
src = req.text

# soup it
soup = BeautifulSoup(src, "lxml")

# Create empty list to store our data
members = []

# returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

# loop through all rows
for row in rows:
    # select only those 'td' tags with class 'detail'
    detailCells = row.select('td.detail')
    
    # get rid of junk rows
    if len(detailCells) is not 5: 
        continue
        
    # Keep only the text in each of those cells
    rowData = [cell.text for cell in detailCells]
    
    # Collect information
    name = rowData[0]
    district = int(rowData[3])
    party = rowData[4]
    
    # add href
    href = row.select('a')[1]['href']
    
    # add full path
    full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"
    
    # Store in a tuple
    tup = (name,district,party, full_path)
    
    # Append to list
    members.append(tup)

In [ ]:
members[:5]

## Challenge 4: Make a function

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator. 

In [ ]:
# SOLUTION
def get_members(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src, "lxml")
    rows = soup.select('tr')
    members = []
    for row in rows:
        detailCells = row.select('td.detail')
        if len(detailCells) is not 5:
            continue
        rowData = [cell.text for cell in detailCells]
        name = rowData[0]
        district = int(rowData[3])
        party = rowData[4]
        href = row.select('a')[1]['href']
        full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"
        tup = (name,district,party,full_path)
        members.append(tup)
    return(members)

In [ ]:
# Test you code!
senateMembers = get_members('http://www.ilga.gov/senate/default.asp?GA=98')
len(senateMembers)

# Part 3: Scrape Bills
*****

## 3.1 Writing a Scraper Function

Now we want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given Bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
I've started the function for you. Fill in the rest.

In [ ]:
# SOLUTION
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src, "lxml")
    rows = soup.select('tr tr tr')
    bills = []
    rowData = []
    for row in rows:
        detailCells = row.select('td.billlist')
        if len(detailCells) is not 5:
            continue
        rowData = [cell.text for cell in row]
        bill_id = rowData[0]
        description = rowData[2]
        champber = rowData[3]
        last_action = rowData[4]
        last_action_date = rowData[5] 
        tup = (bill_id,description,champber,last_action,last_action_date)
        bills.append(tup)
    return(bills)

In [ ]:
# uncomment to test your code:
test_url = senateMembers[0][3]
get_bills(test_url)[0:5]

## 3.2 Get all the bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list_of_bills (the value) eminating from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

NOTE: please call the function `time.sleep(0.5)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# SOLUTION
bills_dict = {}
for member in senateMembers[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(0.5)

In [ ]:
bills_dict[52]